In [1]:

import cv2
from PIL import Image, ImageDraw, ImageFont, ImageTransform
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from glob import glob
import sys

import os
from configparser import ConfigParser
path_config_file = os.getcwd().split('docrv2_sroie')[0] + 'docrv2_sroie/' + 'config.ini'
common_variable = ConfigParser()
common_variable.read(path_config_file)
print(common_variable.sections())
print(common_variable['database']['module_path'])
sys.path.append(common_variable['database']['module_path'])
from db_util import *
from db import *
from random import choice
from cyclegan import CycleGAN
import tensorflow as tf

['database', 'mlflow', 'font']
/home/dk/docrv2_sroie/modules/database/
['database', 'mlflow', 'font']
/home/dk/docrv2_sroie/modules/database/


Using TensorFlow backend.


'style_real_00020_A'

In [8]:
# set simulator parameter
modeldir = '/home/dk/docrv2_sroie/modules/simulator/cyclegan/models/'
modelpath = glob(modeldir+'*_BA.h5')
modelpath.append('None')
def load_model(path):
    if path is not 'None':
        gan = CycleGAN(dataset_name='inference')
        gan.g_AB.load_weights(path)
        return gan
    else:
        return 'None'

def break_model(gan):
    if gan is not 'None':
        tf.keras.backend.clear_session()
        del gan

def inference_batch(gan, batch):
    if gan is not 'None':
        batch = np.array(batch)/127.5-1.
        batch = gan.g_AB.predict(batch)
        batch = 0.5*batch+0.5
        batch = np.uint8(batch*255)
    return batch

In [56]:
# apply cycleGAN later here
path = '/home/dk/docrv2_sroie/DB/'
name = 'denoise_train_100000_nostyle'
db_data = open_env(path+name)
img_raw = read_bulk_data_from_db(db_data,'img')
imgs = []
for idx in range(0,len(img_raw),2):
    imgs.append(img_raw[idx])


In [4]:
# some generator
def make_batch_image(imgs, batch_size):
    numImages = len(imgs)
    numbatch = int(np.ceil(numImages/batch_size))
    imgsize = imgs[0].size

    batchImages = np.zeros((batch_size,imgsize[1],imgsize[0],3), dtype=np.uint8)
    bidx = 0
    for img in imgs:
        if bidx == batch_size:
            batchImages = np.zeros((batch_size,imgsize[1],imgsize[0],3), dtype=np.uint8)
            bidx = 0
        batchImages[bidx,:] = np.array(img, dtype=np.uint8)
        bidx = bidx + 1
        if bidx == batch_size:
            yield batchImages


In [ ]:
batch_size = 100
numImages = len(imgs)
numbatch = int(np.ceil(numImages/batch_size))
batchgen = make_batch_image(imgs, batch_size)
imglist = []
for cidx in tqdm(range(numbatch)):
    batchim = next(batchgen)
    gan = load_model(choice(modelpath))
    
    # apply cyclegan
    batchim_inf = inference_batch(gan, batchim)
        
    # make image list
    for bidx in range(batch_size):
        img = Image.fromarray(batchim_inf[bidx,])
        imglist.append(img)

    # remove model
    break_model(gan)


  0%|          | 0/701 [00:00<?, ?it/s]


  0%|          | 1/701 [00:02<24:18,  2.08s/it]


  0%|          | 2/701 [00:03<23:29,  2.02s/it]


  1%|▏         | 10/701 [00:19<22:54,  1.99s/it]

In [ ]:
#save image to db
ind = 0
for idx in range(0,len(img_raw),2):
    img_raw[idx] = imglist[ind]
    ind = ind + 1
    
#name = name+'_styled'
name = 'denoise_train_100000_styled'
cache = {}
db_data = create_data_db(path=path, name=name, description='datadb_denoising',data_class='denoising')
db_data_db = db_data.open_db(b'db_data')
for idx in tqdm(range(len(img_raw))):
    img = img_raw[idx]
    cache = update_cache(cache, index=idx, label='denoising', ref=name, image=img)
    if idx % 5000 == 0:
        write_cache_to_db(db_data, db_data_db, cache)
        cache = {}
        
write_cache_to_db(db_data, db_data_db, cache)
cache = {}

In [ ]:
print_env(db_data)
refresh_main_db()